<a href="https://colab.research.google.com/github/iamgrossman/alnur.abulgazin/blob/master/hypernym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

!pip install  wikipedia


In [0]:
import pandas as pd
import wikipedia


In [0]:
import os

In [0]:
from tqdm import tqdm


In [0]:
from joblib import Parallel, delayed


In [0]:
import multiprocessing

In [0]:
import networkx as nx


In [0]:
import matplotlib.pyplot as plt
from collections import defaultdict
from ipywidgets import interact, interactive, fixed, interact_manual,widgets
from IPython.display import display


In [0]:
import json


In [0]:
num_cores = multiprocessing.cpu_count()

In [24]:
print(num_cores)
wikipedia.set_lang("ru")
# DATA_PATH_LIST = ['D:','src2','taxonomy-enrichment','data','training_data']
from google.colab import drive 


2


In [25]:
!pip install yargy


     |████████████████████████████████| 51kB 3.8MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 7.1MB 14.2MB/s 


In [0]:
import warnings
wikipedia.set_lang("ru")

from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

import warnings
warnings.filterwarnings('ignore')

warnings.filterwarnings('ignore')

START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),

)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
)

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или'),
        ),
        eq('другие')
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    )
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)


In [0]:
def get_hyperonyms(main_word):
    HYPONYM = eq(utils.deaccent(main_word))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM)
    )
    parser = Parser(RULE) 
    text = utils.deaccent(wikipedia.summary(main_word))
    print(text)
    text = re.sub(r'\(.+?\)', '', text)
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        print(k)


In [29]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [36]:
import pandas as pd

CHARGE_NUMBER = 65

df = pd.read_csv("/content/gdrive/My Drive/lab2_words.csv")
charge = df.loc[df['IN CHARGE NUMBER'] == CHARGE_NUMBER]
charge.head(5)


,Заполнено,Unnamed: 1,IN CHARGE NUMBER,IN CHARGE NAME,HYPONYM,HYPERNYM,HYPERNYM.1,HYPERNYM.2,HYPERNYM.3,HYPERNYM.4,HYPERNYM.5,HYPERNYM.6,HYPERNYM.7,HYPERNYM.8,HYPERNYM.9,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,#ERROR!
1368,NaN,1369,65,Alnur Abulgazin,ФИТОФТОРОЗ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1369,NaN,1370,65,Alnur Abulgazin,ФЛАНЕЦ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1370,NaN,1371,65,Alnur Abulgazin,ФЛЕГМОНА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1371,NaN,1372,65,Alnur Abulgazin,ФЛЕЙТИСТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1372,NaN,1373,65,Alnur Abulgazin,ФЛОРБОЛ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
for index, row in charge.iterrows():
    try:
      print(row['HYPONYM'])
      get_hyperonyms(row['HYPONYM'])
      print("-"*50)
    except: 
        pass


ФИТОФТОРОЗ
Фитофтороз, или фитофтора (от др.-греч. φυτον — «растение» + др.-греч. φθαρτικος — «разрушающии, уничтожающии, гибельныи») — заболевание растении.
--------------------------------------------------
ФЛАНЕЦ
Фланец (от нем. Flansch) — плоская деталь квадратнои, круглои, или инои формы с отверстиями для болтов и шпилек, служащая для прочного (узлы длинных строительных конструкции, например, ферм, балок и др.) и герметичного соединения труб, трубопроводнои арматуры, присоединения труб друг к другу, к машинам, аппаратам и емкостям, для соединения валов и других вращающихся деталеи (фланцевое соединение).
--------------------------------------------------
ФЛЕГМОНА
Флегмона (от др.-греч. φλεγμονη — жар, воспаление) — острое разлитое гноиное воспаление клетчаточных пространств; в отличие от абсцесса, не имеет четких границ.
--------------------------------------------------
ФЛЕЙТИСТ
Флеита (лат. flauta, от flatus — дуновение) — лабиальныи духовои музыкальныи инструмент, в котором пер